# Programmatic access and analysis of the MICrONS dataset

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
<b> Electron Microscopy (EM) data enables morphological reconstruction of neurons and detection of their synaptic connectivity </b>. The <a href=https://www.microns-explorer.org/cortical-mm3>MICrONS dataset </a> is one of the largest datasets volume EM datasets currently available, and spans all layers of visual cortex. We will be using this dataset to query the connectivity between the excitatory neurons in the visual cortex. 

<ol> The exercises will cover: 
    <li>connectivity of individual neurons</li>
    <li>connectivity of inhibitory neurons</li>
    <li>in/out degree of inhibitory neurons.</li>
</ol>

    
<em> Note on data access: </em> To make our lifes easier, we already queried the most of the data needed for this exercise from the database. We have made it available as versioned files that can be read with pandas. The entire dataset is hosted using the <a href=https://caveconnectome.github.io/sections/cave_overview.html> Connectome Annotation Versioning Engine (CAVE) </a>. A separate notebook shows how to use CAVE to generate the files used in this notebook. 
    
    
</div>

In [ ]:
mat_version = 1300

# Paths to compiled data for this workshop
syn_data_url = f"https://github.com/sdorkenw/MICrONS_workshop_CNC_forum/raw/refs/heads/main/data/synapses_proofread_V1_neurons_{mat_version}.feather"
cell_data_url = f"https://github.com/sdorkenw/MICrONS_workshop_CNC_forum/raw/refs/heads/main/data/cell_info_{mat_version}.feather" 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
<b> The CAVEclient</b> is a python library that facilitates communication with a CAVE system. For convenience, we also use the package <b>skeleton_plot</b> which handles rendering the precomputed skeletons. Both packages can be installed with pip.
    
</div>

In [ ]:
!pip install -q skeleton_plot
!pip install -q caveclient 

In [ ]:
import caveclient
import skeleton_plot as skelplot

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
<b> CAVE account setup </b>

<p>In order to manage server traffic, every user needs to create a CAVE account and download a user token to access CAVE's services programmatically. The CAVE infrastructure can be read about in <a href='https://www.biorxiv.org/content/10.1101/2023.07.26.550598v1'>more detail on our preprint</a>. The MICrONS data is publicly available which means that no extra permissions need to be given to a new user account to access the data. Bulk downloads of some static data are also available without an account on <a href='https://microns-explorer.org/'> MICrONs Explorer</a>.

<b> A Google account (or Google-enabled account) is required to create a CAVE account.

<p> Go to: <a href=https://global.daf-apis.com/auth/api/v1/user/token> https://global.daf-apis.com/auth/api/v1/user/token </a>to view a list of your existing tokens
<p> If you have never made a token before: 
<ol>
  <li>go here: <a href=https://minnie.microns-daf.com/materialize/views/datastack/minnie65_public>https://minnie.microns-daf.com/materialize/views/datastack/minnie65_public</a> to accept terms of service</li>
  <li>then go here <a href=https://global.daf-apis.com/auth/api/v1/create_token> https://global.daf-apis.com/auth/api/v1/create_token</a> to create a new token.</li>
</ol>

</div>

In [ ]:
my_token = "PASTE_TOKEN_HERE"
client = caveclient.CAVEclient("minnie65_public", auth_token=my_token)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
<p>If you are running this on your local machine rather than colab, you may also store the token on your machine. This makes future access easier as you do not have to specify the token.

</div>

In [ ]:
# client.auth.save_token(token=my_token, overwrite=True)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
## Proofreading and data quality
Understanding this variablity in data quality is critical when interpretting the MICrONS data.

Automated segmentation of neuronal processes in dense EM imaging is challenging at the size of entire neurons, which can have millimeters of axons and dendrites. The automated segmentation algorithms used in the EM data for this project are not perfect, and so proofreading is necessary to obtain accurate reconstructions of a cell and confidence in the connectivity

In the MICrONS dataset, the general rule is that dendrites onto cells with a cell body are sufficiently proofread to trust synaptic connections onto a cell. Axons on the other hand require so much proofreading that only ~1,650 cells have proofread axons.

Axon and dendrite compartment status are marked separately, as proofreading effort was applied differently to the different compartments in some cells.  In all cases, a status of `TRUE` indicates that false merges have been comprehensively removed, and the compartment is at least ‘clean’. Consult the ‘strategy’ column if completeness of the compartment is relevant to your  research.

Some cells were extended to different degrees of completeness, or with different research goals in mind. This is denoted by 'strategy_axon', which may be one of:

<ul>
    <li>none: No cleaning, and no extension, and status is `FALSE`. </li>
    <li>axon_partially_extended: The axon was extended outward from the soma, following each branch to its termination. Output synapses represent a sampling of potential partners. </li>
    <li>axon_interareal: The axon was extended with a preference for branches that projected to other brain areas. Some axon branches were fully extended, but local connections may be incomplete. Output synapses represent a sampling of potential partners. </li>
    <li>axon_fully_extended: Axon was extended outward from the soma, following each branch to its termination. After initial extension, every endpoint was identified, manually inspected, and extended again if possible. Output synapses represent a largely complete sampling of partners.. </li>
</ul>

<b> For this workshop, we treat all cells with at least `axon_partially_extended` as equally trustworth.</b> This may not be a safe assumption for all analysis, and we are happy to provide more guidance depending on the research question.
    
</div>

In [ ]:
cell_df = pd.read_feather(cell_data_url)

In [ ]:
cell_df["proofreading_strategy"].value_counts()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
## Synapse information

While synapses are stored as any other CAVE table in the database, in this case `synapses_pni_2`, this table is much larger than any other table at more than <b>337 million rows</b>, and it works best when queried in a directed way.

For today's workshop we have collected all of the input and output synapses for the proofread cells in version 1300. When accessing CAVE, the `synapse_query()` function allows you to query the synapse table in a more convenient way than most other tables. In particular, the pre_ids and post_ids let you specify which root id (or collection of root ids) you want to query, with pre_ids indicating the collection of presynaptic neurons and post_ids the collection of postsynaptic neurons.


Note that synapse queries always return the list of every synapse between the neurons in the query, even if there are multiple synapses between the same pair of neurons.

A common pattern to generate a list of connections between unique pairs of neurons is to group by the root ids of the presynaptic and postsynaptic neurons and then count the number of synapses between them. 
    
    
</div>

In [ ]:
syn_df = pd.read_feather(syn_data_url)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
## Neuron morphology: plotting skeletons
    
</div>

In [ ]:
skel_color_map = {3: "firebrick", 4: "salmon", 2: "steelblue", 1: "olive"}

def plot_skeleton(sk_d: dict, plot_soma=True, ax=None):
    """Plots a skeleton.
    
    Args:
        sk: skeleton dict as returned from CAVEclient
        
    Returns: 
        ax: plot axes
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(7, 10), dpi=150)
    
    skelplot.plot_tools.plot_verts(np.array(sk_d["vertices"]), np.array(sk_d["edges"]), radius=np.array(sk_d["radius"]) / 1000 * 1,
                                   ax=ax, skel_colors = sk_d["compartment"], title = "", skel_alpha = 1., 
                                   line_width = 5, x = 'x', y = 'y',  plot_soma = plot_soma, soma_node = sk_d["root"],
                                   color = 'darkslategray', soma_size = 120, invert_y = True, 
                                   skel_color_map = skel_color_map, x_min_max = None, 
                                   y_min_max = None, capstyle = "round", joinstyle = "round",)

    ax.spines['right'].set_visible(False) 
    ax.spines['left'].set_visible(False) 
    ax.spines['top'].set_visible(False) 
    ax.spines['bottom'].set_visible(False)
    ax.axis('off')
    
    return ax

In [ ]:
# select from the cells which are fully extended
root_id = cell_df[cell_df["proofreading_strategy"] == "axon_fully_extended"].iloc[3]["pt_root_id"]

# Query the skeleton from CAVEclient
sk_d = client.skeleton.get_skeleton(root_id, output_format="dict")

In [ ]:
# Plot skeleton in 2D
ax = plot_skeleton(sk_d)
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
## Neuron morphology & synapses
    
</div>

In [ ]:
def filter_synapse_table(synapse_table: pd.DataFrame, pre_root_ids=None, post_root_ids=None):   
    """Filter synapse table by pre and post root ids.

    Args:
        synapse_table: synapse table with pre_pt_root_ids and post_pt_root_ids as pd.DataFrame
        pre_root_ids: np.ndarray, list or pd.Series if root_ids to filter on the presynaptic side
        post_root_ids: np.ndarray, list or pd.Series if root_ids to filter on the postsynaptic side

    Returns:
        synapse_table: filtered synapse table
    """
    
    if pre_root_ids is not None:
        assert isinstance(pre_root_ids, (np.ndarray, list, pd.core.series.Series)), f"IDs have to be of type np.ndarray, list or pd.Series; got {type(pre_root_ids)}"
        pre_m = np.isin(synapse_table["pre_pt_root_id"], pre_root_ids)
    else:
        pre_m = np.ones(len(synapse_table), dtype=bool)
        
    if post_root_ids is not None:
        assert isinstance(post_root_ids, (np.ndarray, list, pd.core.series.Series)), f"IDs have to be of type np.ndarray, list or pd.Series; got {type(pre_root_ids)}"
        post_m = np.isin(synapse_table["post_pt_root_id"], post_root_ids)
    else:
        post_m = np.ones(len(synapse_table), dtype=bool)
        
    return synapse_table[pre_m & post_m]

In [ ]:
# Select the synapses where your root_id is the presynaptic ('upstream') cell; the outputs of your cell
pre_syns = filter_synapse_table(syn_df, pre_root_ids=[root_id])

In [ ]:
pre_syns.head(5)

In [ ]:
# Plot your skeleton and overlay synapses
ax = plot_skeleton(sk_d)

sns.scatterplot(data=pre_syns, x="ctr_pt_position_x", y="ctr_pt_position_y", 
                s=5, color="b", ax=ax, edgecolor=None, zorder=100) 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1:</b> Use the filter function above to create a dataframe of all postsynaptic synapses and create a version of the plot above that includes these postsynaptic synapses. 
    
</div>

In [ ]:
# Select the synapses where your root_id is postsynaptic ('downstream'); the inputs to your cell
post_syns = filter_synapse_table(syn_df, post_root_ids=[root_id])

In [ ]:
# Plot skeleton with input and output synapses
ax = plot_skeleton(sk_d)

sns.scatterplot(data=pre_syns, x="ctr_pt_position_x", y="ctr_pt_position_y", 
                s=5, color="b", ax=ax, edgecolor=None, zorder=100) 

sns.scatterplot(data=post_syns, x="ctr_pt_position_x", y="ctr_pt_position_y", 
                s=5, color="r", ax=ax, edgecolor=None, zorder=100) 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Synapse matrix

The synapses of neurons create a network of synaptic connections. One way of visualizing this connectivity is in a matrix form. Pandas provides the `pivot_table` function that we can use to make a matrix out of the tabular synapse data. For now, we will limit ourselves to the synapses between the proofread neurons.
    
Each synapses has a `size` value assigned to it. How to aggregate the sizes from multiple synapes between two neurons depends on the research question. Synapse sizes vary by a lot and are related to the physiological strength of a synapse ([Holler et al., 2021](https://www.nature.com/articles/s41586-020-03134-2)). Here, we first ignore the synapse size and only look at binary (connected yes/no) connectivity.

</d>

In [ ]:
# get the ids of all proofread cells from the cell_df
proof_root_ids = np.array(cell_df[cell_df["proofreading_strategy"] != ""]["pt_root_id"])

# filter for synapses between the proofread cells (pre and post)
proof_proof_syn_table = filter_synapse_table(syn_df, pre_root_ids=proof_root_ids, post_root_ids=proof_root_ids)

syn_mat = proof_proof_syn_table.pivot_table(index="pre_pt_root_id", columns="post_pt_root_id", 
                                            values="size", aggfunc=lambda x: float(np.sum(x) > 0)).fillna(0)

# Make sure matrix is quadratic
syn_mat = syn_mat.reindex(columns=np.array(syn_mat.index))

In [ ]:
# Plot the connectivity matric between proofread cells
fig, ax = plt.subplots(figsize=(7, 5), dpi=150)
sns.heatmap(syn_mat, cmap="gray_r", xticklabels=[], yticklabels=[], 
            ax=ax, square=True,
            cbar_kws={"label": "Connected - binary"})

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 2:</b> The matrix above only contains connections between proofread neurons. Because dendrites in of cells in the MICrONS dataset are generally well reconstructed we can analyze many more connections than just the ones between proofread neurons. Therefore, the connectivity matrix of analyzable connections forms a rectangular matrix. What is the size of the matrix between the proofread neurons and all neurons with a cell body in the dataset?
    
</div>

In [ ]:
# Select synapses from proofread cells onto all cells
proof_root_ids = np.array(cell_df[cell_df["proofreading_strategy"] != ""]["pt_root_id"])
nuc_root_ids = np.array(cell_df["pt_root_id"])

proof_proof_syn_table = filter_synapse_table(syn_df, pre_root_ids=proof_root_ids, post_root_ids=nuc_root_ids)

rect_syn_mat = proof_proof_syn_table.pivot_table(index="pre_pt_root_id", columns="post_pt_root_id", 
                                            values="size", aggfunc=lambda x: float(np.sum(x) > 0)).fillna(0)

In [ ]:
rect_syn_mat.shape

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

How to take synapse size and number into account depends on the specific analysis. The sizes in the MICrONS dataset measure the synaptic cleft volume annotated by the automated classifier in voxels (3d pixels). These are correlated to anatomical measures such as synaptic area and spine head volumes (for excitatory synapses). Let's replot the square matrix with the average of synapses sizes between each connected pair.

</d>

In [ ]:
proof_root_ids = np.array(cell_df[cell_df["proofreading_strategy"] != ""]["pt_root_id"])
proof_proof_syn_table = filter_synapse_table(syn_df, pre_root_ids=proof_root_ids, post_root_ids=proof_root_ids)

# matrix of mean synapse size
syn_mat = proof_proof_syn_table.pivot_table(index="pre_pt_root_id", columns="post_pt_root_id", 
                                            values="size", aggfunc=lambda x: np.mean(x)).fillna(0)

# Make sure matrix is quadratic
syn_mat = syn_mat.reindex(columns=np.array(syn_mat.index))

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5), dpi=150)
sns.heatmap(syn_mat, cmap="gray_r", xticklabels=[], yticklabels=[], 
            ax=ax, square=True, vmin=0, vmax=1e4,
            cbar_kws={"label": "Mean synapse size (vx)"})

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Depending on the connection type, number of synapses might be the more important measure (see [Dorkenwald et al., 2022](https://elifesciences.org/articles/76120) for deeper dive into synapse size and counts). Next we plot the number of synapses per connection (we limit the colorbar to a maximum of 5 but the actual max number is much higher):


</d>

In [ ]:
proof_root_ids = np.array(cell_df[cell_df["proofreading_strategy"] != ""]["pt_root_id"])
proof_proof_syn_table = filter_synapse_table(syn_df, pre_root_ids=proof_root_ids, post_root_ids=proof_root_ids)

# matrix of sum of synapse size
syn_mat = proof_proof_syn_table.pivot_table(index="pre_pt_root_id", columns="post_pt_root_id", 
                                            values="size", aggfunc=lambda x: np.sum(x > 0)).fillna(0)

# Make sure matrix is quadratic
syn_mat = syn_mat.reindex(columns=np.array(syn_mat.index))

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5), dpi=150)
sns.heatmap(syn_mat, cmap="gray_r", xticklabels=[], yticklabels=[], 
            ax=ax, square=True, vmin=0, vmax=5,
            cbar_kws={"label": "Number of synapses"})

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Cell type tables

Identifying the putative ‘cell type’ from the EM morphology is a process that involves both manual and automatic classifications. Subsets of the dataset have been manually classified by anatomists at the Allen Institute, and these ground truth labels used to train and refine different automated ‘feature classifiers’ over time. 

The diversity of manual and automated cell type classifications available in the dataset reflect the fact that definitions of ‘cell types’ in the dataset is an active area of research and must be contextualized against the purpose and resolution of the cell-typing being performed.

<b> Manual Cell Types (V1 Column)</b>

A subset of nucleus detections in a 100 um column (n=2204) in VISp were manually classified by anatomists at the Allen Institute into categories of cell subclasses, first distinguishing cells into classes of non-neuronal, excitatory and inhibitory. Excitatory cells were separated into laminar sub-classes (L23, L4), 3 sub-types of layer 5 cells (ET, IT, NP) and 2 classes of layer 6 cells (IT, CT). Inhibitory cells were classified into Bipolar (BPC), Basket (BC), Martinotti (MC), or Unsure (Unsure). Those neuronal calls are available from the CAVEclient under the table name `allen_v1_column_types_slanted_ref` which references the nucleus id of the cell, and the column `cell_type_manual` in the prepared data for this workshop. 

Non-neuronal manual cells type calls enumerate astrocytes, microglia, pericytes, oligodendrocytes (oligo), and oligodendrocyte precursor cells (OPC), and area available in the table `aibs_column_nonneuronal_ref`.

<b>Automated Cell Type classification (Soma-nucleus model)</b>

Models were trained based upon the manual Column Neuron labels, as described in <a href=https://www.biorxiv.org/content/10.1101/2022.07.20.499976v1>Elabbady et al. BioRxiv 2023.</a> Each nucleus was analyzed for a variety of features, and a model trained on and independent dataset to distinguish neurons from non-neuronal detections. Non-neuron detections include both glial cells and false positive detections. The nucleus segmentation detected 171,818 connected components of nucleus objects, this model detected 82K neurons. Evaluation of this model on 1,316 cells in the volume shows the model has a recall of 99.6% for neurons, and a precision of 96.9%. All nucleus detections and the results of this model can be queried and linked to the cellular segmentation using the CAVEclient with the table name nucleus_neuron_svm. 

Combining those features with the nucleus features we trained a hierachical model on the manual labels to predict cell-classes and sub-classes across a large number of neurons. This is available as CAVE table `aibs_metamodel_celltypes_v661`, and the column `cell_type_auto` in the prepared data for this workshop.
    
</d>

In [ ]:
# Plot cell volume by depth
fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
ax.tick_params(labelsize=14)
sns.scatterplot(data=cell_df, x="volume", y="pt_position_y", size=1, edgecolor=None, alpha=.01, color="k", ax=ax, legend=False)
ax.invert_yaxis()
ax.set_xlabel("Volume ($\mu m^3$)", fontsize=16)
ax.set_ylabel("Depth ($\mu m$)", fontsize=16)
ax.set_xlim(0, 500)
plt.show()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task 3:</b> The differences in volume are not only indicative of cortical layers, they also relate to cell types. Use the automated cell type columns (`coarse_cell_type_auto` and `cell_type_auto`) to color the scatter plot. The `hue` parameter allows you to do that in the seaborn scatter plot.
    
</div>

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
ax.tick_params(labelsize=14)
sns.scatterplot(data=cell_df, x="volume", y="pt_position_y", size=1, edgecolor=None, alpha=.1, color="k", ax=ax, 
                legend=True, hue="coarse_cell_type_auto")
ax.invert_yaxis()
ax.set_xlabel("Volume ($\mu m^3$)", fontsize=16)
ax.set_ylabel("Depth ($\mu m$)", fontsize=16)
ax.set_xlim(0, 500)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
ax.tick_params(labelsize=14)
sns.scatterplot(data=cell_df, x="volume", y="pt_position_y", size=1, edgecolor=None, alpha=.1, color="k", ax=ax, 
                legend=True, hue="cell_type_auto")
ax.invert_yaxis()
ax.set_xlabel("Volume ($\mu m^3$)", fontsize=16)
ax.set_ylabel("Depth ($\mu m$)", fontsize=16)
ax.set_xlim(0, 500)
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Sorting the synapse matrix with cell types
    
Let's combine the synaptic connecitivity with the cell type information. Below we provide logic for sorting a connectivity matrix using a list of labels.    
    
</d>

In [ ]:
def sort_matrix_by_types(mat: pd.DataFrame, 
                         labels: pd.DataFrame, 
                         label_type_col: str = "cell_type_auto", 
                         label_id_col: str = "pt_root_id", 
                         post_labels: pd.DataFrame = None, 
                         post_label_type_col: str = None, 
                         post_label_id_col: str = None):
    """Sorts (synapse) matrix by labels.

    This function assumes a square synapse matrix!

    Args:
        mat: synapse matrix as pandas DataFrame
        labels: DataFrame with labels, e.g. the output of client.materialize.query_table('aibs_metamodel_celltypes_v661')
        label_type_col: column name in labels for cell types
        label_id_col: column name in labels for root ids
        post_labels: DataFrame with labels, e.g. the output of client.materialize.query_table('aibs_metamodel_celltypes_v661')
        post_label_type_col: column name in labels for cell types
        post_label_id_col: column name in labels for root ids

    Returns:
        mat_sorted: sorted matrix
        mat_labels: sorted labels; has the same length as matrix
    """
    
    if post_labels is None:
        post_labels = labels
    if post_label_type_col is None:
        post_label_type_col = label_type_col
    if post_label_id_col is None:
        post_label_id_col = label_id_col
        
    mat_sorted = mat.copy()
    
    pre_mat_labels = np.array(labels.set_index(label_id_col).loc[mat_sorted.index][label_type_col])
    pre_sorting = np.argsort(pre_mat_labels)

    post_mat_labels = np.array(post_labels.set_index(post_label_id_col).loc[mat_sorted.T.index][post_label_type_col])
    post_sorting = np.argsort(post_mat_labels)

    mat_sorted = mat_sorted.iloc[pre_sorting].T.iloc[post_sorting].T

    return mat_sorted, pre_mat_labels[pre_sorting], post_mat_labels[post_sorting]

In [ ]:
# Select the proofread excitatory cells
proof_E_root_ids = np.array(cell_df[np.array(cell_df["proofreading_strategy"] != "") & np.array(cell_df["coarse_cell_type_auto"] == "E")]["pt_root_id"])
proof_proof_syn_table = filter_synapse_table(syn_df, pre_root_ids=proof_E_root_ids, post_root_ids=proof_E_root_ids)

syn_mat = proof_proof_syn_table.pivot_table(index="pre_pt_root_id", columns="post_pt_root_id", 
                                            values="size", aggfunc=lambda x: float(np.sum(x) > 0)).fillna(0)

syn_mat = syn_mat.reindex(columns=np.array(syn_mat.index))

# sort the matrix by cell types to render sensibly in heatmap
syn_mat_ct, syn_mat_cell_types, _ = sort_matrix_by_types(syn_mat, cell_df, label_type_col="cell_type_auto")

In [ ]:
# add colormap for cell type
cts, ct_idx = np.unique(syn_mat_cell_types, return_inverse=True)
ct_colors = plt.get_cmap("tab10")(ct_idx)

fig, ax = plt.subplots(figsize=(7, 5), dpi=150)
sns.heatmap(syn_mat_ct, cmap="gray_r", xticklabels=[], yticklabels=[], 
            ax=ax, square=True,
            cbar_kws={"label": "Connected - binary"})


# Adding row and column colors for cell types
for i, color in enumerate(ct_colors):
    ax.add_patch(plt.Rectangle(xy=(-0.01, i), width=0.01, height=1, color=color, lw=0,
                               transform=ax.get_yaxis_transform(), clip_on=False))

for i, color in enumerate(ct_colors):
    ax.add_patch(plt.Rectangle(xy=(i, 1), height=0.01, width=1, color=color, lw=0,
                               transform=ax.get_xaxis_transform(), clip_on=False))

# add a legend for the cell types
legend_elements = [matplotlib.lines.Line2D([0], [0], color=plt.get_cmap("tab10")(i), label=ct) for i, ct in enumerate(cts)]
plt.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.3, 1), title="cell types")
plt.show()